In [3]:
from param.arg_utils import get_args
from experiment import Experiment
import easydict
import gym
import pygame
import torch as th
from stable_baselines3 import A2C,PPO
from stable_baselines3.common.env_checker import check_env
from model.extractor import ImgStateExtractor
import os
os.environ['CUDA_LAUNCH_BLOCKING']='1'
device = th.device("cuda:0" if th.cuda.is_available() else "cpu")

params = easydict.EasyDict({
        'env':'gym-2d-perception-v1',
        'gaze_method':'LookAhead',
        'render':False,
        'dt':0.1,
        'map_scale':10,
        'map_size':[640,480],
        'agent_number':10,
        'agent_max_speed':20,
        'agent_radius':10,
        'drone_max_speed':40,
        'drone_max_acceleration':15,
        'drone_radius':5,
        'drone_max_yaw_speed':80,
        'drone_view_depth' : 80,
        'drone_view_range': 90,
        'record': False,
        'record_img':False,
        'pillar_number':3
    })

In [ ]:

alg_params = {
    "policy_kwargs": dict(
        net_arch=[512, dict(pi=[256], vf=[256])],
        normalize_images=False,
        features_extractor_class=ImgStateExtractor,
        features_extractor_kwargs=dict(
            cnn_encoder_name="CnnEncoder"
        ),
    ),
}

env = gym.make('gym-2d-perception-v1', params=params) 
check_env(env)
model = PPO(
            "MultiInputPolicy",
            env,
            verbose=1,
            device=device,
            tensorboard_log='./experiment/log/',
            **alg_params
        )
model.learn(total_timesteps=30000)
pygame.display.quit()
vec_env = model.get_env()
vec_env.is_render = True
obs = vec_env.reset()

In [ ]:
model.save('./trained_policy/lookahead.zip')

In [4]:
env = gym.make('gym-2d-perception-v1', params=params) 

model = PPO.load(path='./trained_policy/lookahead.zip')
model.set_env(env)

model.learn(total_timesteps=30000)
pygame.display.quit()
vec_env = model.get_env()
obs = vec_env.reset()

Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Logging to ./experiment/log/PPO_6


/home/smoggy/anaconda3/envs/thesis/lib/python3.9/site-packages/gym/spaces/box.py:73: UserWarning: WARN: Box bound precision lowered by casting to float32
  logger.warn(
/home/smoggy/anaconda3/envs/thesis/lib/python3.9/site-packages/stable_baselines3/common/base_class.py:719: UserWarning: You are probably loading a model saved with SB3 < 1.7.0, we deactivated exact_match so you can save the model again to avoid issues in the future (see https://github.com/DLR-RM/stable-baselines3/issues/1233 for more info). Original error: Error(s) in loading state_dict for MultiInputActorCriticPolicy:
	Missing key(s) in state_dict: "pi_features_extractor.cnn_encoder_single.0.cnn.0.weight", "pi_features_extractor.cnn_encoder_single.0.cnn.0.bias", "pi_features_extractor.cnn_encoder_single.0.cnn.2.weight", "pi_features_extractor.cnn_encoder_single.0.cnn.2.bias", "pi_features_extractor.cnn_encoder_single.0.cnn.4.weight", "pi_features_extractor.cnn_encoder_single.0.cnn.4.bias", "pi_features_extractor.cnn_en

---------------------------------
| rollout/           |          |
|    ep_len_mean     | 331      |
|    ep_rew_mean     | 868      |
| time/              |          |
|    fps             | 25       |
|    iterations      | 1        |
|    time_elapsed    | 79       |
|    total_timesteps | 2048     |
---------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 337         |
|    ep_rew_mean          | 757         |
| time/                   |             |
|    fps                  | 29          |
|    iterations           | 2           |
|    time_elapsed         | 140         |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.010382827 |
|    clip_fraction        | 0.103       |
|    clip_range           | 0.2         |
|    entropy_loss         | -1.41       |
|    explained_variance   | 0.723       |
|    learning_rate        | 0.

In [ ]:
params = easydict.EasyDict({
        'env':'gym-2d-perception-v1',
        'gaze_method':'LookAhead',
        'render':True,
        'dt':0.1,
        'map_scale':10,
        'map_size':[640,480],
        'agent_number':10,
        'agent_max_speed':20,
        'agent_radius':10,
        'drone_max_speed':40,
        'drone_max_acceleration':15,
        'drone_radius':5,
        'drone_max_yaw_speed':80,
        'drone_view_depth' : 80,
        'drone_view_range': 90,
        'record': False,
        'record_img': False,
        'pillar_number':3
    })
env = gym.make('gym-2d-perception-v1', params=params) 
obs = env.reset()
for i in range(2000):
    action, _state = model.predict(obs, deterministic=True)

    obs, reward, done, info = env.step(action)
    env.render()

    if done:
        obs = env.reset()